## Load libraries

In [33]:
import json
import numpy as np
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
from companionCode.wrangling.parse import df_from_bib
from companionCode.visualization.categorical_plots import *
from matplotlib_venn import venn2, venn2_circles, venn3, venn3_circles
from warnings import filterwarnings
filterwarnings('ignore')

## Visualization options

In [16]:
# custom colors
c1_ = [i / 255 for i in [227, 111, 71]]
c2_ = [i / 255 for i in [1, 155, 250]]
c3_ = [i / 255 for i in [63, 164, 77]]
c4_ = [i / 255 for i in [195, 113, 211]]
# rendering option
matplotlib.rcParams["text.usetex"] = True
matplotlib.rcParams["font.family"] = "serif"
matplotlib.rcParams["font.size"] = "11"
# alpha params
alpha = [0.6]

## Scopus search outcome and coding outcome

### Read data

In [17]:
# Scopus's search outcome
df0 = df_from_bib("data/management_studies_on_ex_shocks_clean.bib")
print("Number of retrieved articles: {}".format(len(df0)))
# coded data
df1 = pd.read_csv("data/coded_studies.csv")
print("Number of articles considered in the coding: {}".format(len(df1 )))
# merge data
df = pd.merge(df0, df1.loc[df1["sample"] == True], on="key", how="inner")
print("Number of coded articles: {}".format(len(df)))
# delete data
del df0, df1


Number of retrieved articles: 58
Number of articles considered in the coding: 58
Number of coded articles: 37


### Sample description

#### Distribution of studies across journals

In [18]:
chart = BarChart(name_="manuscript/exhibits/studies_across_journals")
chart.add_data(df_=df, var_="journal")
chart.plot(
    fig_size=(3.75, 2.5),
    use_tex=True,
    edge_color="white",
    face_color="gray",
    tick_min=0,
    tick_step=2,
    axis_label="Counts of studies",
    horizontal_=True,
)



                  PGF is a non-GUI backend -- nothing to display here!
                  
                  To display the chart here, set 'use_tex' = False 
                  


### Type of naturally-occurring event

In [19]:
chart = BarChart(name_="manuscript/exhibits/classes_of_events")
chart.add_data(df_=df, var_="event_class")
chart.plot(
    fig_size=(3.75, 3.5),
    use_tex=True,
    face_color="gray",
    edge_color="white",
    tick_min=0,
    tick_step=2,
    axis_label="Counts of studies",
    horizontal_=True,
)



                  PGF is a non-GUI backend -- nothing to display here!
                  
                  To display the chart here, set 'use_tex' = False 
                  


### Event-research question mapping

In [46]:
# get cross tab
pd.crosstab(df["role_empirical"], df["role_theory"])
# get cross tab
cross_tab = pd.crosstab(df["role_empirical"], df["role_theory"])
# create figure
fig = plt.figure(figsize=(2, 2))
ax = fig.add_subplot(1, 1, 1)
venn2(
    subsets=(cross_tab.iloc[1, 0], cross_tab.iloc[0, 1], cross_tab.iloc[1, 1]),
    set_labels=("Empirical role", "Substantive role"),
    set_colors=(c2_, c1_),
    alpha=0.75,
)
plt.savefig("manuscript/exhibits/event_roles.pgf", bbox_inches="tight", pad_inches=0)


### Potential outcome framework

In [50]:
# get cross tab
df_ = df.loc[df["control_group"] == True]
pd.crosstab(df_["role_empirical"], df_["role_theory"])
# get cross tab
cross_tab = pd.crosstab(df_["role_empirical"], df_["role_theory"])
# create figure
fig = plt.figure(figsize=(2, 2))
ax = fig.add_subplot(1, 1, 1)
venn2(
    subsets=(cross_tab.iloc[1, 0], cross_tab.iloc[0, 1], cross_tab.iloc[1, 1]),
    set_labels=("Empirical role", "Substantive role"),
    set_colors=(c2_, c1_),
    alpha=0.75,
)
plt.savefig("manuscript/exhibits/event_roles_po_yes.pgf", bbox_inches="tight", pad_inches=0)
del df_


In [52]:
# get cross tab
df_ = df.loc[df["control_group"] == False]
pd.crosstab(df_["role_empirical"], df_["role_theory"])
# get cross tab
cross_tab = pd.crosstab(df_["role_empirical"], df_["role_theory"])
# create figure
fig = plt.figure(figsize=(2, 2))
ax = fig.add_subplot(1, 1, 1)
venn2(
    subsets=(cross_tab.iloc[1, 0], cross_tab.iloc[0, 1], cross_tab.iloc[1, 1]),
    set_labels=("Empirical role", "Substantive role"),
    set_colors=(c3_, c4_),
    alpha=0.75,
)
plt.savefig("manuscript/exhibits/event_roles_po_no.pgf", bbox_inches="tight", pad_inches=0)
del df_


In [21]:
# data
# --+ event plays empirical role
gr = df.loc[(df["role_empirical"] == True) & (df["role_theory"] == False)].groupby(
    ["control_group"], as_index=False
)
c0 = pd.DataFrame(gr.size())
# --+ event plays substantive role
gr = df.loc[(df["role_empirical"] == False) & (df["role_theory"] == True)].groupby(
    ["control_group"], as_index=False
)
c1 = pd.DataFrame(gr.size())
# --+ event plays both roles
gr = df.loc[(df["role_empirical"] == True) & (df["role_theory"] == True)].groupby(
    ["control_group"], as_index=False
)
c2 = pd.DataFrame(gr.size())


In [28]:
c2

,control_group,size
0,False,3
1,True,7


In [25]:
# create stacked bar chart
fig = plt.figure(figsize=(2.5, 0.9))
ax = fig.add_subplot(1, 1, 1)
ax.barh(
    c0.index,
    c0["size"],
    height=0.25,
    label="Empirical",
    color="gray"
)
ax.barh(
    c2.index,
    c2["size"],
    left=c0["size"],
    height=0.25,
    label="Empirical \& substantive",
    color="gray",
    alpha=0.7
)
ax.barh(
    c1.index,
    c1["size"],
    height=0.25,
    left=c0["size"] + c2["size"],
    label="Substantive",
    color="gray",
    alpha=0.4
)
# labels and ticks
ax.set_yticks([0, 1])
ax.set_yticklabels(["Design lacking a control group", "Design with a control group"])
ax.set_xlabel("Counts of studies")
# hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(True)
ax.spines["left"].set_visible(False)
ax.yaxis.set_ticks_position("left")
ax.xaxis.set_ticks_position("bottom")
# legend
ax.legend(loc=(1, 0), frameon=False)
# tick labels' jsutification
plt.setp(
    ax.get_yticklabels(),
    rotation=0,
    ha="right",
    va="center",
    rotation_mode="anchor",
)
# plot
plt.savefig("manuscript/exhibits/potential_outcome.pgf", bbox_inches="tight", pad_inches=0)


In [49]:
df.groupby("role_description").size()

role_description
Empirical / attenuating endogeneity concerns                            13
Empirical / attenuating endogeneity concerns (supplemental analysis)     3
Theory testing                                                           9
Theory testing & empirical / attenuating endogeneity concerns           10
Theory testing & empirical / creating measures                           1
dtype: int64

In [50]:
df.groupby("event_class").size()

event_class
Analyst coverage change     2
Data breaches               1
Investigation               1
Legal change               12
Pandemic                    2
Political referendum        1
Price variation             1
Product discontinuation     1
Recession                   1
Scandal                     2
Software piracy             1
Terrorist attack            4
Turnover                    7
dtype: int64